In [18]:
from datasets import load_dataset
import numpy as np
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
import torch
import kornia as K
from PIL import Image

hf_dataset = load_dataset("blanchon/UC_Merced", split= "train")
# ucmerced_test = load_dataset("blanchon/UC_Merced", split= "train[80%:]")

In [ ]:
class PreProcess(torch.nn.Module):
    """Module to perform pre-process using Kornia on torch tensors."""
    def __init__(self) -> None:
        super().__init__()
 
    @torch.no_grad()  # disable gradients for effiency
    def forward(self, x: Image) -> torch.Tensor:
        x_tmp: np.ndarray = np.array(x)  # HxWxC
        x_out: torch.Tensor = K.image_to_tensor(x_tmp, keepdim=True)  # CxHxW
        return x_out.float() / 255.0

train_transforms = torch.nn.Sequential(
    PreProcess(),
    K.augmentation.Resize(size=224, side="short"),
    K.augmentation.CenterCrop(size=224),
    K.augmentation.RandomHorizontalFlip(p=0.5),
    K.augmentation.ColorJiggle(),
    K.augmentation.Normalize(mean=[0.4914, 0.4822, 0.4465], std=[0.2023, 0.1994, 0.2010]),
)

val_transforms = torch.nn.Sequential(
    PreProcess(),
    K.augmentation.Resize(size=224, side="short"),
    K.augmentation.CenterCrop(size=224),
    K.augmentation.Normalize(mean=[0.4914, 0.4822, 0.4465], std=[0.2023, 0.1994, 0.2010]),
)

def preprocess_train(example_batch):
    """Apply train_transforms across a batch."""
    example_batch["image"] = [train_transforms(image).squeeze() for image in example_batch["image"]]
    return example_batch

def preprocess_val(example_batch):
    """Apply val_transforms across a batch."""
    example_batch["image"] = [val_transforms(image).squeeze() for image in example_batch["image"]]
    return example_batch

In [25]:
hf_dataset.set_transform(preprocess_train)

In [29]:
hf_dataset[0]

{'image': <PIL.TiffImagePlugin.TiffImageFile image mode=RGB size=256x256>,
 'label': 0,
 'pixel_values': tensor([[[-0.1223,  0.1933, -0.1395,  ..., -1.4957, -1.4184, -1.5761],
          [ 0.0933, -0.1962, -0.8109,  ..., -1.5360, -1.5481, -1.6315],
          [-0.4044, -0.3578, -0.5113,  ..., -1.5344, -1.5686, -1.6093],
          ...,
          [-0.1811,  0.1930,  0.2250,  ...,  2.0073,  1.4690,  1.2415],
          [ 0.1241,  0.0349, -0.0801,  ...,  1.1014,  0.6602,  0.0501],
          [ 0.1879, -0.5044, -0.4100,  ...,  0.1529, -0.2135,  0.0522]],
 
         [[-0.0779,  0.2451, -0.0813,  ..., -1.5360, -1.4518, -1.6119],
          [ 0.0678, -0.1412, -0.7495,  ..., -1.5704, -1.5835, -1.6681],
          [-0.3583, -0.2040, -0.3428,  ..., -1.5639, -1.6091, -1.6456],
          ...,
          [-0.1629,  0.2524,  0.2687,  ...,  2.0431,  1.5348,  1.2720],
          [ 0.1018,  0.0046, -0.0607,  ...,  1.1379,  0.6558, -0.0266],
          [ 0.1188, -0.5330, -0.3529,  ...,  0.0608, -0.3615, -0.0976]]

In [30]:
train_loader = DataLoader(hf_dataset, batch_size=16, shuffle=True, num_workers=0)

for batch in train_loader:
    print(batch)
    break

TypeError: default_collate: batch must contain tensors, numpy arrays, numbers, dicts or lists; found <class 'PIL.TiffImagePlugin.TiffImageFile'>

In [ ]:
labels = hf_dataset.features["label"].names
print("ALL LABELS")
print(labels)
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = str(i)
    id2label[str(i)] = label

print("\nALL LABELS to ID")
print(label2id)
print("\nALL ID to LABELS")
print(id2label)

In [ ]:
# Inspect an example
dataset[0][0].shape
